# GD with MNIST_revision

## BGD, MBGD, SGD의 비교

- BGD의 장점
    - 전체 데이터에 대해 업데이트가 한번에 이루어지기 때문에 후술할 SGD 보다 업데이트 횟수가 적다. 따라서 전체적인 계산 횟수는 적다.
    - 전체 데이터에 대해 error gradient 를 계산하기 때문에 optimal 로의 수렴이 안정적으로 진행된다.
    - 병렬 처리에 유리하다.
- BGD의 단점
    - 한 스텝에 모든 학습 데이터 셋을 사용하므로 학습이 오래 걸린다.
    - 전체 학습 데이터에 대한 error 를 모델의 업데이트가 이루어지기 전까지 축적해야 하므로 더 많은 메모리가 필요하다.
    - local optimal 상태가 되면 빠져나오기 힘듦(SGD 에서 설명하겠음.)
- SGD의 장점
    - 위 그림에서 보이듯이 Shooting 이 일어나기 때문에 local optimal 에 빠질 리스크가 적다.
    - step 에 걸리는 시간이 짧기 때문에 수렴속도가 상대적으로 빠르다.
- SGD의 단점
    - global optimal 을 찾지 못 할 가능성이 있다.
    - 데이터를 한개씩 처리하기 때문에 GPU의 성능을 전부 활용할 수 없다.
- MBGD의 장점
    - BGD보다 local optimal 에 빠질 리스크가 적다.
    - SGD보다 병렬처리에 유리하다.
    - 전체 학습데이터가 아닌 일부분의 학습데이터만 사용하기 때문에 메모리 사용이 BGD 보다 적다.
- MBGD의 단점
    - batch size(mini-batch size) 를 설정해야 한다.
    - 에러에 대한 정보를 mini-batch 크기 만큼 축적해서 계산해야 하기 때문에 SGD 보다 메모리 사용이 높다.

![](https://www.nomidl.com/wp-content/uploads/2022/08/image-11.png)

## Logistic function definition

In [ ]:
import numpy as np
import matplotlib as mpl
import matplotlib.pyplot as plt
mpl.rcParams['figure.dpi'] = 150

z = np.arange(-5, 5, 0.1)
phi = 1 / (1 + np.exp(-z))

plt.plot(z, phi)
plt.xlabel('z')
plt.ylabel('phi')
plt.title('Sigmoid function')
plt.show()

## Loss function definition
- In linear regression, loss is defined as 
$ \displaystyle \mathcal{L}(w, b) = \frac{1}{2n}\sum_{i=0}^{N-1}(\hat{y_i}-y_i)^2 = \frac{1}{2n}\sum_{i=0}^{N-1}\{(wx_i+b)-y_i\}^2 $.
- To calculate derivatives, we use $ \displaystyle \frac{\partial \mathcal{L}}{\partial w} = \frac{\partial \mathcal{L}}{\partial z} \frac{\partial z}{\partial w} = \frac{1}{n}((wx_i+b)-y_i)x_i$ for sample $i$.
- For classification problem, how do we define loss function?

### Logistic loss function ($y$, $\hat{y}$ is 0 or 1)
- $ \displaystyle \mathcal{L} = -\{y \log(\hat{y}) + (1-y) \log(1-\hat{y})\} $.

### Loss definition according to phi (Logistic loss function)

In [ ]:
mpl.rcParams['figure.dpi'] = 150
fig, axs = plt.subplots(1, 2)

x = np.arange(0.001, 1-0.001, 0.001)
cost_0 = -1 * np.log(1 - x)
cost_1 = -1 * np.log(x)

axs[0].plot(x, cost_0)
axs[1].plot(x, cost_1)
axs[0].set_xlabel('y_hat')
axs[1].set_xlabel('y_hat')
fig.supylabel('Loss')
axs[0].set_title("if y = 0")
axs[1].set_title("if y = 1")

plt.show()

### Previous GD code

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt

def check_accuracy(weights, X, Y):
    correct = 0
    for samplenum in range(X.shape[0]):
        input = X[samplenum].reshape(WIDTH*HEIGHT, 1)
        pred = np.dot(weights, input)
        pred = np.argmax(pred)
        if pred == Y[samplenum]:
            correct += 1
    return correct / X.shape[0]

(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
SCALE_FACTOR = 255
WIDTH = X_train.shape[1]
HEIGHT = X_train.shape[2]
X_train = X_train.reshape(X_train.shape[0],WIDTH*HEIGHT) / SCALE_FACTOR
X_test = X_test.reshape(X_test.shape[0],WIDTH*HEIGHT)  / SCALE_FACTOR

weights = np.random.rand(10, WIDTH*HEIGHT) - 0.5
alpha = 0.01
iteration = 100000

for iter in range(iteration):
    ind = np.random.randint(X_train.shape[0])
    input = X_train[ind].reshape(WIDTH*HEIGHT, 1)
    true = Y_train[ind]
    true = np.array([1 if i == true else 0 for i in range(10)]).reshape(10, 1)
    
    pred = np.dot(weights, input)
    error = (pred - true) ** 2
    delta = pred - true

    weight_deltas = np.outer(delta, input)
    weights -= alpha * weight_deltas

    if iter % np.round(iteration/10) == 0:
        print(f"Iteration: {iter}, \
              Accuracy for train: {check_accuracy(weights, X_train, Y_train)}")

print(f"Accuracy for test: {check_accuracy(weights, X_test, Y_test)}")

## Softmax function
``` python
pred_1 = [[0.9]
          [0.8]
          [0.7]],
pred_2 = [[0.5]
          [0.2]
          [0.1]]
```
Which prediction is better?

- Softmax function --> Output intensity normalization

### Logistic loss function
- $ \displaystyle \mathcal{L} = -\{y \log(\hat{y}) + (1-y) \log(1-\hat{y})\} $.
### Cross-entropy loss function (Generalized version of logistic loss function)
- $ \displaystyle \mathcal{L} = -\sum_{c=1}^C y_c \log(\hat{y}) = -\log(\hat{y}_{y=1})$.

### Improved code with softmax function

In [ ]:
import pandas as pd
import numpy as np
from tensorflow.keras.datasets import mnist
import matplotlib.pyplot as plt

def check_accuracy(weights, X, Y):
    correct = 0
    for samplenum in range(X.shape[0]):
        input = X[samplenum].reshape(WIDTH*HEIGHT, 1)
        pred = np.dot(weights, input)
        pred = np.argmax(pred)
        if pred == Y[samplenum]:
            correct += 1
    return correct / X.shape[0]

(X_train, Y_train), (X_test, Y_test) = mnist.load_data()
SCALE_FACTOR = 255
WIDTH = X_train.shape[1]
HEIGHT = X_train.shape[2]
X_train = X_train.reshape(X_train.shape[0],WIDTH*HEIGHT) / SCALE_FACTOR
X_test = X_test.reshape(X_test.shape[0],WIDTH*HEIGHT)  / SCALE_FACTOR

weights = np.random.rand(10, WIDTH*HEIGHT) - 0.5
alpha = 0.01
iteration = 100000

def sigmoid(x):
    return 1 / (1 + np.exp(-x))

def softmax(x):
    x = np.clip(x, -100, 100)   # overflow 방지
    return np.exp(x) / np.sum(np.exp(x), axis=0)

def cross_entropy(pred, true):
    return -np.sum(true * np.log(pred))

for iter in range(iteration):
    ind = np.random.randint(X_train.shape[0])
    input = X_train[ind].reshape(WIDTH*HEIGHT, 1)
    true = Y_train[ind]
    true = np.array([1 if i == true else 0 for i in range(10)]).reshape(10, 1)
    
    pred = np.dot(weights, input)
    pred = softmax(pred)    # softmax를 적용해준다.
    error = cross_entropy(pred, true)   # cross entropy를 적용해준다.
    delta = pred - true # 사실 이 부분은 굉장히 복잡한데, 여기서는 간단하게 생각하자.

    weight_deltas = np.outer(delta, input)
    weights -= alpha * weight_deltas

    if iter % np.round(iteration/10) == 0:
        print(f"Iteration: {iter}, \
              Accuracy for train: {check_accuracy(weights, X_train, Y_train)}")

print(f"Accuracy for test: {check_accuracy(weights, X_test, Y_test)}")

## Important!
1. 로지스틱 회귀에서 SGD로 진행되는 논리 숙지할 것.
- 1이라고 확신했는데 (즉, z 값이 아주 높았었는데,) 사실 0이었다면?
    - Loss가 아주 높아야 함
- 1이라고 확신했는데 (즉, z 값이 아주 높았었는데,) 역시 1이었다면?
    - Loss가 아주 낮아야 함
2. BGD, MBGD, SGD의 이해
3. Softmax function, cross-entropy loss function의 이해